In [1]:
import pandas as pd

# DOMA
# Define the file path
nodes_path = r"C:\Users\Doktor\Documents\GitHub\Bridges\Data\Nodes_processed.csv"
roads_path = r"C:\Users\Doktor\Documents\GitHub\Bridges\Data\Roads_processed.csv"
bridges_path = r"C:\Users\Doktor\Documents\GitHub\Bridges\Data\Bridges_processed.csv"
bridges_selected_path = r"C:\Users\Doktor\Documents\GitHub\Bridges\Data\Bridges_selected.csv"
survey_path = r"C:\Users\Doktor\Documents\GitHub\Bridges\Data\surveys\Survey.csv"


# # Práca
# # Define the file path
# nodes_path = r"C:\Users\relia\Documents\GitHub\Bridges\Data\Nodes_processed.csv"
# roads_path = r"C:\Users\relia\Documents\GitHub\Bridges\Data\Roads_processed.csv"
# bridges_path = r"C:\Users\relia\Documents\GitHub\Bridges\Data\Bridges_processed.csv"
# bridges_selected_path = r"C:\Users\relia\Documents\GitHub\Bridges\Data\Bridges_selected.csv"
# survey_path = r"C:\Users\relia\Documents\GitHub\Bridges\Data\surveys\Survey.csv"

# Load the CSV data into a DataFrame
df_nodes = pd.read_csv(nodes_path, encoding='utf-8', delimiter=',')
df_roads = pd.read_csv(roads_path, encoding='utf-8', delimiter=',')
df_bridges_all = pd.read_csv(bridges_path, encoding='utf-8', delimiter=',')
df_bridges_sel = pd.read_csv(
    bridges_selected_path, encoding='utf-8', delimiter=',')
df_survey = pd.read_csv(survey_path, encoding='utf-8', delimiter=',')

C:\Users\Doktor\AppData\Local\Temp\ipykernel_10504\4094816067.py:21: DtypeWarning: Columns (5,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_nodes = pd.read_csv(nodes_path, encoding='utf-8', delimiter=',')
C:\Users\Doktor\AppData\Local\Temp\ipykernel_10504\4094816067.py:22: DtypeWarning: Columns (10,11,12,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df_roads = pd.read_csv(roads_path, encoding='utf-8', delimiter=',')


In [2]:
df_bridges_all['Dĺžka_premostenia'] = pd.to_numeric(
    df_bridges_all['Dĺžka_premostenia'], errors='coerce')

df_bridges_all.columns

Index(['ID_mosta', 'Trieda_PK', 'Číslo_PK', 'Most_DC', 'Názov_mosta', 'ID_DC',
       'správcovské_číslo', 'Zemepisná_dĺžka', 'Zemepisná_šírka', 'Výška',
       'Druh_konštrukcie', 'Material', 'Normálna_zaťažiteľnosť',
       'Výhradná_zaťažiteľnosť_t', 'Výnimočná_zaťažiteľnosť_t',
       'Spôsob_stanovenia', 'Predmet_premostenia', 'Rok_postavenia',
       'Zaťažovacia_trieda_STN', 'Počet_otvorov', 'Dĺžka_premostenia',
       'Dĺžka_nosnej_konštrukcie_m', 'Volná_šírka_mosta',
       'Šírka_medzi_obrubami_m', 'Plocha_mosnej_konštrukcie_m2',
       'Plocha_mosta_m2', 'Vlastník_úseku', 'Správca_úseku', 'Okres',
       'Smer_úseku', 'Typ_úseku', 'Počiatočný_uzol', 'Koncový_uzol', 'úsek_ID',
       'úsekové_staničenie_začiatku_DC_m',
       'Kilometrovníkové_staničenie_začiatku_DC_km',
       'Kumulatívne_staničenie_začiatku_DC_km', 'Poradie', 'Kraj', 'n_2012',
       'n_2013', 'n_2014', 'n_2015', 'n_2016', 'n_2017', 'n_2018', 'n_2019',
       'n_2020', 'n_2021', 'n_2022', 'n_2023'],
      

In [3]:
# Filter bridges in the Nitra region with Trieda_PK as "cesta I. triedy" or "cesta II. triedy"
filtered_bridges = df_bridges_all[
    (df_bridges_all['Kraj'] == "Banská Bystrica") &
    (df_bridges_all['n_2023'] > 4) &
    ((df_bridges_all['Trieda_PK'] == "cesta II. triedy") |
     (df_bridges_all['Trieda_PK'] == "cesta III. triedy")) &
    (df_bridges_all['Dĺžka_premostenia'] >= 10) &
    (df_bridges_all['Material'] == "prefabrikovaný predpätý betón")
]

# Display the filtered DataFrame
filtered_bridges.shape

(29, 51)

In [4]:
import geopandas as gpd
# Replace the file path with your actual file path
file_path = r'C:\Users\Doktor\Documents\GitHub\Bridges\GeoJSONs\regions_epsg_4326.geojson'  # doma
# file_path = r'C:\Users\relia\Documents\GitHub\Bridges\GeoJSONs\regions_epsg_4326.geojson' # robota


# Load the GeoJSON file into a GeoDataFrame
gdf = gpd.read_file(file_path)
# Display the first few rows of the GeoDataFrame
# gdf

In [6]:
import folium


def plot_filtered_bridges(df, gdf, regions):
    # Find the average coordinates for the initial map center
    avg_lat = df['Zemepisná_šírka'].mean()
    avg_long = df['Zemepisná_dĺžka'].mean()

    # Create the map with custom tileset
    m = folium.Map(location=[avg_lat, avg_long],
                   zoom_start=9, tiles="Cartodb positron")

    # Colors for the markers
    colors = ["#ff6600", "#0066FF"]

    # Bridge icon
    bridge_icon_url = "https://raw.githubusercontent.com/Medvedku/Assets/main/markers/bridge_marker_Cartodb_orng.svg"

    for index, row in df.iterrows():
        ic_size = 50
        bridge_icon = folium.CustomIcon(
            icon_image=bridge_icon_url,
            icon_size=(ic_size, ic_size),
            icon_anchor=(ic_size/2, ic_size)
        )

        popup_html = f"<table style='width:400px;'>"
        popup_html += f"<tr><th style='text-align:left;'>Názov mosta:</th><td>{row['Názov_mosta']}</td></tr>"
        popup_html += f"<tr><th style='text-align:left;'>Trieda PK:</th><td>{row['Trieda_PK']}</td></tr>"

        popup_html += "<tr><th style='text-align:left;'>Dĺžka premostenia:</th><td>"
        if pd.notna(row['Dĺžka_premostenia']):
            popup_html += f"{float(row['Dĺžka_premostenia'])} m"
        else:
            popup_html += "Unknown"
        popup_html += "</td></tr>"

        popup_html += f"<tr><th style='text-align:left;'>Druh konštrukcie:</th><td>{row['Druh_konštrukcie']}</td></tr>"
        popup_html += f"<tr><th style='text-align:left;'>Materiál:</th><td>{row['Material']}</td></tr>"
        popup_html += f"<tr><th style='text-align:left;'>Rok postavenia:</th><td>{row['Rok_postavenia']}</td></tr>"
        popup_html += f"<tr><th style='text-align:left;'>GPS Coordinates:</th><td>({row['Zemepisná_šírka']:.4f}, {row['Zemepisná_dĺžka']:.4f})</td></tr>"
        popup_html += f"</table>"

        popup = folium.Popup(popup_html, parse_html=False)

        tooltip_html = f"<table style='width:50px;'>"
        tooltip_html += f"<tr><th colspan='2' style='text-align:left;'>Most {row['ID_mosta']}</th></tr>"
        tooltip_html += f"<tr><td style='text-align:left;'>STS:</td><td>{row['n_2023']}</td></tr>"
        tooltip_html += f"</table>"

        # Add the marker with the custom icon and popup
        folium.Marker(
            [row['Zemepisná_šírka'], row['Zemepisná_dĺžka']],
            icon=bridge_icon,
            popup=popup,
            tooltip=tooltip_html
        ).add_to(m)

    # Filter the GeoDataFrame to include only the specified regions
    region_gdf = gdf[gdf['NM4'].isin(regions)]

    # Define custom style for the regions' polygons
    def style_function(x): return {
        'fillColor': '#FF6600', 'color': '#FF6600', 'weight': 2}

    # Add the boundaries of the specified regions to the map with the custom style
    folium.GeoJson(region_gdf, name='Regions Boundaries',
                   style_function=style_function).add_to(m)

    return m


# Usage example (replace 'filtered_bridges' with your filtered DataFrame and adjust regions as needed)
regions_to_plot = ['Banskobystrický']
map = plot_filtered_bridges(filtered_bridges, gdf, regions_to_plot)
map.save('filtered_bridges.html')
map

In [7]:
filtered_bridges[["ID_mosta", "Názov_mosta",
                  'n_2023', 'Material', "Dĺžka_premostenia"]]

,ID_mosta,Názov_mosta,n_2023,Material,Dĺžka_premostenia
2297,M7079,524-12 Most nad poľnou cestou v Banskej Štiavn...,5,prefabrikovaný predpätý betón,70.14
2313,M4237,526-20_Most cez potok Slatina za obcou Korytár...,6,prefabrikovaný predpätý betón,17.50
2386,M4048,000527-037_Most cez potok Krupinica v obci Zábava,6,prefabrikovaný predpätý betón,14.00
2965,M2219,000571-032 Most cez rieku Rimava v obci Jesenské,6,prefabrikovaný predpätý betón,24.90
2970,M3780,000571-039 Most cez potok Blh za obcou Rim...,7,prefabrikovaný predpätý betón,14.70
3266,M7269,000591-033 Most cez potok Tisovník v obci Doln...,6,prefabrikovaný predpätý betón,17.40
3869,M7392,"2480-001 Most cez potok Lutila v km 0,539 Jano...",5,prefabrikovaný predpätý betón,14.00
3885,M5488,"2484-001 Most cez Lutilský potok v km 0,676 Ži...",6,prefabrikovaný predpätý betón,17.70
3932,M4242,2694-2 Most cez rieku Slatina pri železničnej...,6,prefabrikovaný predpätý betón,17.88
4006,M2601,2753-6 Most cez potok Blh v obci Veľký Blh (0...,6,prefabrikovaný predpätý betón,14.70


In [8]:
filtered_bridges["n_2023"].value_counts()

n_2023
5    15
6    12
7     2
Name: count, dtype: int64